In [ ]:
!pip install -q sentinelsat
!pip install -q rasterio
!apt install -q gdal-bin python-gdal python3-gdal 
!apt install -q python3-rtree 
!pip install -q descartes 
!pip install -q shapely
!pip install -q six
!pip install -q pyproj
#!pip install git+git://github.com/geopandas/geopandas.git
!pip install -q geopandas

     |████████████████████████████████| 48 kB 3.1 MB/s 
     |████████████████████████████████| 19.3 MB 1.4 MB/s 
Reading package lists...
Building dependency tree...
Reading state information...
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 22 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-

In [ ]:
import os
import numpy as np

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt 
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from shapely.geometry import MultiPolygon, Polygon
import rasterio as rio
from rasterio.plot import show
import rasterio.mask
import fiona
import folium 
from osgeo import gdal
from geojson import Polygon
import zipfile
from PIL import Image
import json

In [ ]:
user = '**********' 
password = '**********' 
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')


In [ ]:

def getpolygon(lat,lon): #34.4,135,4
  poly = [
      [lon,     lat+0.4],
      [lon+0.4, lat+0.4],
      [lon+0.4, lat],
      [lon,     lat],
      [lon,     lat+0.4],
  ]
  return Polygon([poly])

In [ ]:
!mkdir output

In [ ]:
def getimage(lat,lon, numtry):
  m = getpolygon(lat, lon)
  #ファイル名を定義. 好きな名称に設定してください.
  object_name0 = str(lat) + str(lon)
  with open(str(object_name0) +'.geojson', 'w') as f:
      json.dump(m, f)
  footprint_geojson = geojson_to_wkt(read_geojson(str(object_name0) +'.geojson'))
  products = api.query(footprint_geojson,
                     date = ('20220101', '20221010'), #取得希望期間の入力
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,100)) #被雲率（0％〜100％）
  products_gdf = api.to_geodataframe(products)
  print(len(products_gdf))
  products_gdf_sorted = products_gdf[products_gdf["ondemand"] ==  "false"] #データがオンラインなもののみを抽出
  products_gdf_sorted = products_gdf_sorted[products_gdf_sorted["size"].str.contains('GB')] #1GB越えのもののみを抽出
  products_gdf_sorted = products_gdf_sorted.sort_values(['cloudcoverpercentage'], ascending=[True]) #被雲率でソート
  j = 0
  for i in range(len(products_gdf_sorted)):
    try:
      object_name = str(lat) + str(lon) + "-" + str(i)
      print(products_gdf_sorted.iloc[i]["beginposition"])
      uuid = products_gdf_sorted.iloc[i]["uuid"]
      product_title = products_gdf_sorted.iloc[i]["title"]
      api.download(uuid)
      file_name = str(product_title) +'.zip'
      with zipfile.ZipFile(file_name) as zf:
        zf.extractall()

      path = str(product_title) + '.SAFE/GRANULE'
      files = os.listdir(path)

      pathA = str(product_title) + '.SAFE/GRANULE/' + str(files[0])
      files2 = os.listdir(pathA)
      files2[1] = 'IMG_DATA'
      pathB = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2[1]) +'/R10m'
      files3 = os.listdir(pathB)
      path_b2 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2[1]) +'/R10m/' +str(files3[0][0:23] +'B02_10m.jp2')
      path_b3 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2[1]) +'/R10m/' +str(files3[0][0:23] +'B03_10m.jp2')
      path_b4 = str(product_title) + '.SAFE/GRANULE/' + str(files[0]) +'/' + str(files2[1]) +'/R10m/' +str(files3[0][0:23] +'B04_10m.jp2')
      b4 = rio.open(path_b4)
      b3 = rio.open(path_b3)
      b2 = rio.open(path_b2)
      with rio.open(str(object_name) +'.tiff','w',driver='Gtiff', width=b4.width, height=b4.height, 
                count=3,crs=b4.crs,transform=b4.transform, dtype=b4.dtypes[0]) as rgb:
        rgb.write(b2.read(1),3) 
        rgb.write(b3.read(1),2) 
        rgb.write(b4.read(1),1) 
        rgb.close()

      RGB_tokyo =rio.open(str(object_name) +'.tiff')

      nReserve_geo = gpd.read_file(str(object_name0) +'.geojson')
      epsg = b4.crs
      nReserve_proj = nReserve_geo.to_crs({'init': epsg})

      with rio.open(str(object_name) +'.tiff') as src:
          out_image, out_transform = rio.mask.mask(src, nReserve_proj.geometry,crop=True)
          out_meta = src.meta.copy()
          out_meta.update({"driver": "GTiff",
                      "height": out_image.shape[1],
                      "width": out_image.shape[2],
                      "transform": out_transform})
          
      with rasterio.open('Masked_' +str(object_name) +'.tif', "w", **out_meta) as dest:
          dest.write(out_image)

      msk = rio.open(r'Masked_' +str(object_name) +'.tif')
      scale = '-scale 0 255 0 10'
      options_list = [
          '-ot Byte',
          '-of JPEG',
          scale
      ] 
      options_string = " ".join(options_list)

      gdal.Translate('./output/sentinel2-' +str(object_name) +'.jpg',
                    'Masked_' +str(object_name) +'.tif',
                    options=options_string)
      j +=1
    except Exception as e:
      print(e)
    if j >= numtry:
      break

In [ ]:
getimage(lat,lon,3)


2022-01-10 01:40:29.024000


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


2022-06-04 01:37:01.024000


MD5 checksumming:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


2022-10-02 01:37:01.024000


MD5 checksumming:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
for li in range(3):
  lat = 33.6 + 0.4*li
  for lon in [135.4,135.8]:
    print(lon)
    getimage(lat,lon,2)

135.4


Querying products:  89%|########9 | 100/112 [00:00<?, ?product/s]

112
2022-06-04 01:37:01.024000


MD5 checksumming:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


...

Querying products:  91%|######### | 100/110 [00:00<?, ?product/s]

110
2022-01-10 01:40:29.024000


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


2022-01-10 01:40:29.024000


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
im = Image.open('Masked_' +str(object_name) +'.jpg')
im

In [ ]:
!zip -r output.zip output

  adding: output/ (stored 0%)
  adding: output/sentinel2-33.6135.4-1.jpg.aux.xml (deflated 62%)
  adding: output/sentinel2-34.4135.4-0.jpg.aux.xml (deflated 62%)
  adding: output/sentinel2-34.0135.4-0.jpg (deflated 0%)
  adding: output/.ipynb_checkpoints/ (stored 0%)
  adding: output/sentinel2-34.0135.8-1.jpg.aux.xml (deflated 62%)
  adding: output/sentinel2-34.4135.4-0.jpg (deflated 0%)
  adding: output/sentinel2-34.0135.8-0.jpg (deflated 0%)
  adding: output/sentinel2-34.4135.8-1.jpg.aux.xml (deflated 62%)
  adding: output/sentinel2-33.6135.8-0.jpg.aux.xml (deflated 62%)
  adding: output/sentinel2-33.6135.4-0.jpg (deflated 1%)
  adding: output/sentinel2-34.0135.4-1.jpg.aux.xml (deflated 62%)
  adding: output/sentinel2-34.0135.8-0.jpg.aux.xml (deflated 62%)
  adding: output/sentinel2-33.6135.8-1.jpg.aux.xml (deflated 62%)
  adding: output/sentinel2-33.6135.8-1.jpg (deflated 1%)
  adding: output/sentinel2-34.4135.4-1.jpg (deflated 0%)
  adding: output/sentinel2-34.0135.4-0.jpg.aux.xml 